In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dropout
from keras.models import Model
import random

In [2]:
np.random.seed(0)

In [3]:
(X_train, y_train), (X_test, y_test)= mnist.load_data()
 
print(X_train.shape)
print(X_test.shape)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (28,28)), "The dimensions of the images are not 28 x 28."
assert(X_test.shape[0] == y_test.shape[0]), "The number of images is not equal to the number of labels."
assert(X_test.shape[1:] == (28,28)), "The dimensions of the images are not 28 x 28."

(60000, 28, 28)
(10000, 28, 28)


In [4]:
num_of_samples=[]
 
cols = 5
num_classes = 10
 
fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(5,10))
fig.tight_layout()
 
for i in range(cols):
    for j in range(num_classes):
      x_selected = X_train[y_train == j]
      axs[j][i].imshow(x_selected[random.randint(0,(len(x_selected) - 1)), :, :], cmap=plt.get_cmap('gray'))
      axs[j][i].axis("off")
      if i == 2:
        axs[j][i].set_title(str(j))
        num_of_samples.append(len(x_selected))

In [5]:
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the train dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")
plt.show()

[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]


In [6]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
 
X_train = X_train/255
X_test = X_test/255

In [8]:
# lenet dataseti tanımlıyorum.
def lenet_model():
  model = Sequential()
  model.add(Conv2D(30, (5,5), input_shape=(28,28,1), activation='relu', strides=1, padding='same'))
  # modelde shapei verdiğimiz resmin pixellerine göre veriyorum, stride kaç kere gösterildiğini, padding resmi ne kadar küçülyüp,büyüyeceğini gösteriyor.
  # modele filter ekliyorum istediğim kadar ekleyebilirim.Efektive olarak 30 tane filter kullanılıyor lenette.Daha fazlası yapılır ama fazla GPU gerekli.
  model.add(MaxPooling2D(pool_size=(2,2)))
  #max pooling size bana ulaşacabileceği max sizeı istiyor, ben değişmesini istemediğimden tuple olarak veriyorum.
  model.add(Conv2D(15, (3,3), activation='relu'))
  # activation kullanıcağımız tekniği belirtir.
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  # oluşacak matrisi one dimensional hale getirmek için kullandığım fonksiyon.
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  # güncellemeden sonra oluşan fazlalığı dropout ile alıyorum.
  # Dense ile görünmeyen layerları ekliyorum.
  model.add(Dense(num_classes, activation='softmax'))
  model.compile(Adam(lr = 0.01), loss="categorical_crossentropy", metrics=['accuracy'])
  return model

In [9]:
model = lenet_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 30)        780       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 15)        4065      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 15)          0         
_________________________________________________________________
flatten (Flatten)            (None, 540)               0         
_________________________________________________________________
dense (Dense)                (None, 500)               270500    
_________________________________________________________________
dropout (Dropout)            (None, 500)               0

In [10]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=400, verbose=1, shuffle= 1)
# verbose progress bar için, 1 veya 0 değeri verilir sadece.
#shuffle datanın çalışması için 0 veremeyiz.

Epoch 1/10
135/135 [==============================] - 19s 140ms/step - loss: 0.2306 - accuracy: 0.9254 - val_loss: 0.0557 - val_accuracy: 0.9847

In [11]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss','val_loss'])
plt.title('Loss')
plt.xlabel('epoch')

Text(0.5, 0, 'epoch')

In [12]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')

Text(0.5, 0, 'epoch')

In [13]:
import requests 
from PIL import Image
url = 'https://colah.github.io/posts/2014-10-Visualizing-MNIST/img/mnist_pca/MNIST-p1815-4.png'
response = requests.get(url, stream=True)
print(response)
img = Image.open(response.raw)
plt.imshow(img)

<Response [200]>


In [14]:
from cv2 import *

img_array = np.asarray(img)
resized = cv2.resize(img_array, (28, 28))
gray_scale = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
image = cv2.bitwise_not(gray_scale)
#bitwise ile birlikte fotoğrafta ne kadar boşkuk varsa yani 0 lar 2:55 le dolduruluyor.yani aslında siyah olan yer beyaz 
#beyaz olan siyah oluyor. Böylelikle image i daha rahat yakalıyoruz.
plt.imshow(image, cmap=plt.get_cmap("gray"))
print(image)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 210]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 210]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 210]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  67 170
  254 155 100  17   0   0   0   0   0 210]
 [  0   0   0   0   0   0   0   0   0   0   0  18  89 176 155 198 239 252
  252 252 252 174   0   0   0   0   0 209]
 [  0   0   0   0   0   0   0   0   0   0  27 234 251 252 142  28  28  28
   62 222 252 246  87   0   0   0   0 210]
 [  0   0   0   0   0   0   0   0   0   0  28 228 243 229 119   2   2   2
   18 185 252 247  88   0   0   0   0 210]
 [  0   0   0   0   0   0   0   0   0   0   0   0  69   0   0   

In [15]:
image = image/255
image = image.reshape(1, 28, 28, 1)
print(image)

[[[[0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.82352941]]

  [[0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [0.        ]
   [

In [16]:
prediction = model.predict_classes(img)
print("predicted digit:", str(prediction))


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


ValueError: Failed to find data adapter that can handle input: <class 'PIL.PngImagePlugin.PngImageFile'>, <class 'NoneType'>

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(type(score))
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
layer1 = Model(inputs=model.layers[0].input, outputs=model.layers[0].output)
layer2 = Model(inputs=model.layers[0].input, outputs= model.layers[2].output)

visual_layer1, visual_layer2 = layer1.predict(img), layer2.predict(img)
print(visual_layer1.shape)
print(visual_layer2.shape)

In [ ]:
plt.figure(figsize=(10,6))
for i in range(30):
  plt.subplot(6, 5, i+1)
  plt.imshow(visual_layer1[0, :, : i], cmap=plt.get_cmap('jet'))
  plt.axis('off')

In [ ]:
plt.figure(figsize=(10,6))
for i in range(15):
  plt.subplot(3, 5, i+1)
  plt.imshow(visual_layer2[0, :, : i], cmap=plt.get_cmap('jet'))
  plt.axis('off')

In [ ]:
plt.figure(figsize=(10,6))
for i in range(15):
  plt.subplot(3, 5, i+1)
  plt.imshow(visual_layer2[0, :, : i], cmap=plt.get_cmap('jet'))
  plt.axis('off')

In [ ]:
(X_train, y_train), (X_test, y_test)= mnist.load_data()
 
print(X_train.shape)
print(X_test.shape)
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (28,28)), "The dimensions of the images are not 28 x 28."
assert(X_test.shape[0] == y_test.shape[0]), "The number of images is not equal to the number of labels."
assert(X_test.shape[1:] == (28,28)), "The dimensions of the images are not 28 x 28."

In [ ]:
num_of_samples=[]
 
cols = 5
num_classes = 10
 
fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(5,10))
fig.tight_layout()
 
for i in range(cols):
    for j in range(num_classes):
      x_selected = X_train[y_train == j]
      axs[j][i].imshow(x_selected[random.randint(0,(len(x_selected) - 1)), :, :], cmap=plt.get_cmap('gray'))
      axs[j][i].axis("off")
      if i == 2:
        axs[j][i].set_title(str(j))
        num_of_samples.append(len(x_selected))

In [ ]:
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the train dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")
plt.show()

In [ ]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
 
X_train = X_train/255
X_test = X_test/255

In [ ]:
# lenet dataseti tanımlıyorum.
def lenet_model():
  model = Sequential()
  model.add(Conv2D(30, (5,5), input_shape=(28,28,1), activation='relu', strides=1, padding='same'))
  # modelde shapei verdiğimiz resmin pixellerine göre veriyorum, stride kaç kere gösterildiğini, padding resmi ne kadar küçülyüp,büyüyeceğini gösteriyor.
  # modele filter ekliyorum istediğim kadar ekleyebilirim.Efektive olarak 30 tane filter kullanılıyor lenette.Daha fazlası yapılır ama fazla GPU gerekli.
  model.add(MaxPooling2D(pool_size=(2,2)))
  #max pooling size bana ulaşacabileceği max sizeı istiyor, ben değişmesini istemediğimden tuple olarak veriyorum.
  model.add(Conv2D(15, (3,3), activation='relu'))
  # activation kullanıcağımız tekniği belirtir.
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  # oluşacak matrisi one dimensional hale getirmek için kullandığım fonksiyon.
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  # güncellemeden sonra oluşan fazlalığı dropout ile alıyorum.
  # Dense ile görünmeyen layerları ekliyorum.
  model.add(Dense(num_classes, activation='softmax'))
  model.compile(Adam(lr = 0.01), loss="categorical_crossentropy", metrics=['accuracy'])
  return model

In [ ]:
model = lenet_model()
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=400, verbose=1, shuffle= 1)
# verbose progress bar için, 1 veya 0 değeri verilir sadece.
#shuffle datanın çalışması için 0 veremeyiz.

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss','val_loss'])
plt.title('Loss')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')

In [ ]:
import requests 
from PIL import Image
url = 'https://colah.github.io/posts/2014-10-Visualizing-MNIST/img/mnist_pca/MNIST-p1815-4.png'
response = requests.get(url, stream=True)
print(response)
img = Image.open(response.raw)
plt.imshow(img)

In [ ]:
from cv2 import *

img_array = np.asarray(img)
resized = cv2.resize(img_array, (28, 28))
gray_scale = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
image = cv2.bitwise_not(gray_scale)
#bitwise ile birlikte fotoğrafta ne kadar boşkuk varsa yani 0 lar 2:55 le dolduruluyor.yani aslında siyah olan yer beyaz 
#beyaz olan siyah oluyor. Böylelikle image i daha rahat yakalıyoruz.
plt.imshow(image, cmap=plt.get_cmap("gray"))
print(image)

In [ ]:
image = image/255
image = image.reshape(1, 28, 28, 1)
print(image)

In [ ]:
prediction = model.predict_classes(img)
print("predicted digit:", str(prediction))


In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(type(score))
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
layer1 = Model(inputs=model.layers[0].input, outputs=model.layers[0].output)
layer2 = Model(inputs=model.layers[0].input, outputs= model.layers[2].output)

visual_layer1, visual_layer2 = layer1.predict(img), layer2.predict(img)
print(visual_layer1.shape)
print(visual_layer2.shape)

In [ ]:
plt.figure(figsize=(10,6))
for i in range(30):
  plt.subplot(6, 5, i+1)
  plt.imshow(visual_layer1[0, :, : i], cmap=plt.get_cmap('jet'))
  plt.axis('off')

In [ ]:
plt.figure(figsize=(10,6))
for i in range(15):
  plt.subplot(3, 5, i+1)
  plt.imshow(visual_layer2[0, :, : i], cmap=plt.get_cmap('jet'))
  plt.axis('off')